# [모듈 2.1] Triton Docker 에 한개의 NCF 모델 서빙

# 1. 환경 셋업

## 1.1. 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

전 노트북에서 훈련 후의 아티펙트를 가져옵니다.

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

## 1.2. 배포 준비

### 이전 노트북에서 훈련된 모델의 S3 경로 확인

In [ ]:
%store -r artifact_path

In [ ]:
print("model artifact is assigend from : ", artifact_path)

### 추론을 위한  데이터 세트 로딩
- 전부 데이터를 로딩할 필요가 없지만, 여기서는 기존에 사용한 함수를 이용하기 위해서 전체 데이터를 로드 합니다. 


In [ ]:
import data_utils 
train_data, test_data, user_num ,item_num, train_mat = data_utils.load_all(test_num=100)

### 파라미터 생성
- 모델 로딩시에 아라 파라미터 사용 

In [ ]:
class Params:
    def __init__(self):
        # self.epochs = 1        
        self.num_ng = 4
        self.batch_size = 256
        self.test_num_ng = 99
        self.factor_num = 32
        self.num_layers = 3
        self.dropout = 0.0
        # self.lr = 0.001
        self.top_k = 10
        self.out = True
        # self.gpu = "0"
                        
args = Params()
print("# of batch_size: ", args.batch_size)


# 2. 훈련된 모델 아티펙트 다운로드 및 압축해제
- 모델 아티펙트를 다운로드 합니다.
- 다운로드 받은 모델 아티펙트의 압축을 해제하고 모델 가중치인 models/model.pth 파일을 얻습니다.

In [ ]:
import os
import config

model_data_dir = config.model_path
os.makedirs(model_data_dir, exist_ok=True)
print("model_data_dir: ", model_data_dir)

In [ ]:
%%sh -s {artifact_path} {model_data_dir}

artifact_path=$1
model_data_dir=$2

echo $artifact_path
echo $model_data_dir

# 기존 데이터 삭제
rm -rf $model_data_dir/*

# 모델을 S3에서 로컬로 다운로드
aws s3 cp $artifact_path $model_data_dir

# 모델 다운로드 폴더로 이동
cd $model_data_dir

# 압축 해제
tar -xvf model.tar.gz  

# 3. 훈련된 모델 로딩


## 3.1. 모델 네트워크 설정 저장
- 모델 네트워크를 생성시에 사용할 설정값을 model_config.json 로 저장함.
- model_fn() 함수에서 모델 네트워크를 생성시에 사용 함.

In [ ]:
import json
from common_utils import save_json, load_json

model_config_dict = {
    'user_num': str(user_num),
    'item_num': str(item_num),
    'factor_num' : str(args.factor_num),
    'num_layers' : str(args.num_layers),
    'dropout' : str(args.dropout),
    'model_type': config.model
}

model_config_file = 'model_config.json'
model_config_file_path = os.path.join('src', model_config_file)

save_json(model_config_file_path, model_config_dict)
# model_config_dict = load_json(model_config_file_path)    
# model_config_dict

## 3.2. 모델 로딩
- 복수개의 모델로 진행하기 위해서, 편의상 동일한 모델에서 생성 함.


In [ ]:
from inference import model_fn

ncf_food_model = model_fn(config.model_path)


In [ ]:
ncf_food_model

# 4. Trition 서빙 준비

## 4.1. 샘플 입력 생성

In [ ]:
import numpy as np
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

user_np = np.zeros((1,100)).astype(np.int32)
item_np = np.random.randint(low=1, high=1000, size=(1,100)).astype(np.int32)

dummy_inputs = [
    torch.from_numpy(user_np).to(device),
    torch.from_numpy(item_np).to(device)
]
print("dummy_inputs: \n", dummy_inputs)
dummy_user = dummy_inputs[0] 
dummy_item = dummy_inputs[1] 

# dummy_inputs

## 4.2. 샘플 입력으로 모델 추론 테스트

In [ ]:
result = ncf_food_model(dummy_user, dummy_item)
print("result shape: ", result.shape)
# result = ncf_fashion_model(dummy_user, dummy_item)
# print("result shape: ", result.shape)

## 4.3. Torch Script 으로 변환

In [ ]:
is_trace = True
is_script = False

In [ ]:
def trace_model(mode, device, model, dummy_inputs, trace_model_name):

    model = model.eval()
    model.to(device)

    if mode == 'trace' :
        IR_model = torch.jit.trace(model, dummy_inputs)

    elif mode == 'script':
        IR_model = torch.jit.script(model)

    print(f"As {mode} : Model is saved {trace_model_name}")
    torch.jit.save(IR_model, trace_model_name)

    print("#### Load Test ####")    
    loaded_m = torch.jit.load(trace_model_name)    
    print(loaded_m.code)    
    dummy_user = dummy_inputs[0]
    dummy_item = dummy_inputs[1]    
    
    result = loaded_m(dummy_user, dummy_item)
    print("Result shape: ", result.shape)

        
if is_trace:
    mode = 'trace'    
elif is_script:    
    mode = 'script'

# food
trace_food_model_name = 'ncf_food_model.pt'    
trace_model(mode, device, ncf_food_model, dummy_inputs, trace_food_model_name)    


## 4.4.config.pbtxt 생성

### ncf_food_config 생성

In [ ]:
%%writefile ncf_food_config.pbtxt
name: "ncf_food_model"
platform: "pytorch_libtorch"
max_batch_size: 128
input [
  {
    name: "INPUT__0"
    data_type: TYPE_INT32
    dims: [100]
  },
  {
    name: "INPUT__1"
    data_type: TYPE_INT32
    dims: [100]
  }
]
output [
  {
    name: "OUTPUT__0"
    data_type: TYPE_FP32
    dims: [-1]
  }
]


# 5. 아티펙트 패키징

## 싱글 모델
- 아래와 닽은 폴더 구조를 생성해야 함.
```
model_serving_folder
    - model_name
        - version_number
            - model file
        - config file
        
# Example: 

triton-serve-pt
    - ncf_food
        - 1
            - model.pt
        - config.pbtxt

```


## 5.1. ncf_food_model 폴더 생성 및 아티펙트 카피

In [ ]:
import os
from triton_util import make_folder_structure, copy_artifact, remove_folder


In [ ]:
# ncf_food_model 폴더 생성
model_serving_folder = 'triton-docker-serve-pt'
model_name = 'ncf_food_model'
make_folder_structure(model_serving_folder, model_name)

fodd_config = 'ncf_food_config.pbtxt'
copy_artifact(model_serving_folder, model_name, trace_food_model_name, fodd_config)

### 폴더 삭제
- 필요시 주석 제거하고 사용하세요.

In [ ]:
# model_serving_folder = 'triton-docker-serve-pt'
# remove_folder(model_serving_folder)

# 6. 로컬 도커에서 실행 테스트

## 6.0. 도커에서의 실행 테스트는 아래와 같은 순서로 진행 함.

#### (0) Triton Client 초기화
```
from triton_util import setup_triton_client
triton_client, grpcclient = setup_triton_client()
```

#### (1) 터미널 실행
![terminal.png](img/terminal.png)

#### (2) Triton 도커 컨테이너 실행
- 위의 터미널에 아래와 같이 명령어를 하나씩 실행 하세요.
```
cd /home/ec2-user/SageMaker/Neural-Collaborative-Filtering-On-SageMaker/2_Triton_Inference

docker run --gpus=1 --rm -p8000:8000 -p8001:8001 -p8002:8002 -v `pwd`/triton-docker-serve-pt:/models nvcr.io/nvidia/tritonserver:22.08-py3 tritonserver --model-repository=/models --log-verbose=3 --log-info=1 --log-warning=1 --log-error=1
```
#### (3) Triton 클라이언트로 추론 실행
#### (4) 도커 중단 및 삭제
```
docker rm -f $(docker ps -qa)
```

## 6.1. Triton Client 초기화

In [ ]:
from triton_util import setup_triton_client
triton_client, grpcclient = setup_triton_client()

## 6.2. !!! #### 터미널에 "Triton 도커 컨테이너 실행" 을 해주세요. ### !!!

## 6.3. 입력 payload 생성

In [ ]:
def create_client_payload():
    inputs = []

    inputs.append(grpcclient.InferInput('INPUT__0', [1,100], "INT32"))
    inputs.append(grpcclient.InferInput('INPUT__1', [1,100], "INT32"))

    # user
    input0_data = np.zeros((1,100)).astype(np.int32)
    inputs[0].set_data_from_numpy(input0_data)

    # item
    input1_data = np.random.randint(low=1, high=1000, size=(1,100)).astype(np.int32)
    inputs[1].set_data_from_numpy(input1_data)

    print("input0_data: \n",input0_data) 
    print("input1_data: \n",input1_data) 
    
    return inputs


In [ ]:
inputs = create_client_payload()


## 6.4. 출력 변수 생성

In [ ]:
outputs = []
outputs.append(grpcclient.InferRequestedOutput('OUTPUT__0'))


## 6.5. Triton에 추론 요청
- 추론 요청이 오면 서버에서 처맇하여 결과를 내보냄. 서버에서의 처리 내용 로그를 한번 보세요.
![single_triton_server_log.png](img/single_triton_server_log.png)

In [ ]:
from triton_util import infer_triton_client

model_name = "ncf_food_model"
infer_triton_client(triton_client, model_name, inputs, outputs)

# 7. 변수 저장

In [ ]:
%store model_serving_folder
%store model_name